In [1]:
import os
import json
from datetime import datetime
import pytz
import pandas as pd

%pip install tabulate

Note: you may need to restart the kernel to use updated packages.


## constants

In [2]:
INPUT_FOLDER = "./input"
OUTPUT_FOLDER = "./output"

FACILITIES_FILE = "au_facilities.json"

MISSING_DATES_FILE = "missing_dates.csv"

TIME_ZONE = pytz.timezone("Australia/Sydney")

FACILITIES_URL = 'https://data.dev.opennem.org.au/v3/geo/au_facilities.json'

## process the facilities geoJSON

### load facilities

In [3]:
import requests

# check to see if  INPUT_FILE is in the INPUT_FOLDER and if not, load it from FACILITIES_URL
def load_facilities_file():
    input_file = os.path.join(INPUT_FOLDER, FACILITIES_FILE)
    if os.path.exists(input_file):
        print(f"INFO: input file {input_file} found")
    else:
        # Ensure the directory exists
        directory = os.path.dirname(input_file)
        if not os.path.exists(directory):
            os.makedirs(directory)

        print(f"WARN: input file {input_file} not found. loading from {FACILITIES_URL}")
        response = requests.get(FACILITIES_URL)

        # Check if the request was successful
        if response.status_code == 200:
            with open(input_file, 'wb') as file:
                file.write(response.content)
            print(f"INFO: file downloaded successfully and saved as {input_file}")
        else:
            print(f"ERROR: failed to download file. status code: {response.status_code}")

load_facilities_file()

WARN: input file ./input/au_facilities.json not found. loading from https://data.dev.opennem.org.au/v3/geo/au_facilities.json
INFO: file downloaded successfully and saved as ./input/au_facilities.json


In [4]:
# read and parse the input file
from pandas import NaT

def read_input(input_file):
    with open(input_file) as f:
        data = json.load(f)
    return data

# given a date string in format "2024-06-10T06:50:00Z" parse it and format like "30 May 2024"
# if the input string is empty, return "**unknown**"
def format_date(date_string):
    if date_string is None:
        return "**unknown**"
    else:
        return datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%SZ").strftime("%d %b %Y")

# iterate through all facilities and units within the facility. 
# for each unit add a pointer to the facility to the unit
def add_parent_pointer(data):
    for facility in data['features']:
        for unit in facility['properties']['duid_data']:
            unit['parent_station_code'] = facility['properties']['station_code']

def build_unit_string(unit):
    # build an operating date string: if operating it should be "since YYYY-MM-DD"ArithmeticError
    # if retired it should be "retired (from YYYY-MM-DD to YYYY-MM-DD)"
    # format dates as DD MMM YYYY
    if unit['status'] == 'operating':
        operating_date = f"since {format_date(unit['data_first_seen'])}"
    elif unit['status'] == 'retired':
        operating_date = f"from {format_date(unit['data_first_seen'])} to {format_date(unit['data_last_seen'])}"
    else:
        operating_date = "unknown"
    
    # put all on a single line for readability
    return f"{unit['parent_station_code']}:{unit['duid']} {unit['capacity_registered']} MW {unit['fuel_tech']} {unit['status']} {operating_date}"


# in the object tree (from au_facilities.json), iterate through each facility and print a summary of the
# DUIDs in each (its capacity in MW, its fuel technology, and its status)
def print_summary(data):
    for facility in data['features']:
        print(facility['properties']['name'])
        for unit in facility['properties']['duid_data']:
           print("  ", build_unit_string(unit)) 
        print()

# determine whether a unit has missing dates
# if the unit has a status of "operating" it needs a data_first_seen date
# if the unit has a status of "retired" it needs a data_first_seen date and a data_last_seen date

def has_missing_dates(status, data_first_seen, data_last_seen):
    if data_first_seen == NaT:
        data_first_seen = None
        
    if data_last_seen == NaT:
        data_last_seen = None

    if status == 'operating':
        return data_first_seen is None
    elif status == 'retired':
        return data_first_seen is None or data_last_seen is None
    elif status in {'commissioning', 'committed'}:
        return False
    else:
        # throw an error
        raise ValueError(f"Unknown status: {status}")

def has_missing_dates_units(unit):
    return has_missing_dates(unit['status'], unit['data_first_seen'], unit['data_last_seen'])   

# function to build a list of units iwth missing dates
def log_units_with_missing_dates(data):
    missing_dates = []
    for facility in data['features']:
        for unit in facility['properties']['duid_data']:
            if has_missing_dates_units(unit):
                missing_dates.append(unit)


    # if there are any units with missing dates
    if len(missing_dates) > 0:
        missing_dates_file = os.path.join(OUTPUT_FOLDER, MISSING_DATES_FILE)
        print(f"WARN: {len(missing_dates)} units have missing dates. writing to {MISSING_DATES_FILE}")

        # make a dataframe of the missing dates and write it to a CSV file
        os.makedirs(OUTPUT_FOLDER, exist_ok=True)
        missing_dates_df = pd.DataFrame(missing_dates)

        # filter to just  duid, fuel_tech, status, capacity_registered, data_first_seen, data_last_seen, network_region, parent_station_code
        missing_dates_df = missing_dates_df[['duid', 'fuel_tech', 'status', 'capacity_registered', 'data_first_seen', 'data_last_seen', 'network_region', 'parent_station_code']]
        missing_dates_df.to_csv(missing_dates_file, index=False)
    else:
        print("INFO: congrautlations! no units have missing dates!")

# print some simple stats
def print_stats(data):
    print(f"There are {len(data['features'])} facilities.")
    print(f"There are {sum([len(f['properties']['duid_data']) for f in data['features']])} units.")
    print(f"There are {len(set([u['status'] for f in data['features'] for u in f['properties']['duid_data']]))} unique status codes:")
    # print the status codes
    for status in set([u['status'] for f in data['features'] for u in f['properties']['duid_data']]):
        print(f" * {status}")


def load_and_parse_units():
    # build the path to the input file
    input_file = os.path.join(INPUT_FOLDER, FACILITIES_FILE)
    data = read_input(input_file)

    add_parent_pointer(data)
    log_units_with_missing_dates(data)
    print_stats(data)

    return data

# call the function
raw_units_data = load_and_parse_units()


WARN: 200 units have missing dates. writing to missing_dates.csv
There are 481 facilities.
There are 728 units.
There are 4 unique status codes:
 * committed
 * operating
 * retired
 * commissioning


In [5]:
# find the earliest data_first_seen date in the list of units
def find_first_seen(units):
    first_seen = units['data_first_seen'].dropna().min()
    return first_seen


# build a pandas data from of units
# with the following columns:
# facility_code, duid, network_region, capacity_registered, fuel_tech, status, data_first_seen, data_last_seen
def build_unit_data_frame(data):
    rows = []
    for facility in data['features']:
        for unit in facility['properties']['duid_data']:
            # parse the data_first_seen and data_last_seen dates
            if unit['data_first_seen'] is not None:
                first_seen = datetime.strptime(unit['data_first_seen'], "%Y-%m-%dT%H:%M:%SZ")
            else:
                first_seen = None
                
            if unit['data_last_seen'] is not None:
                last_seen = datetime.strptime(unit['data_last_seen'], "%Y-%m-%dT%H:%M:%SZ")
            else:
                last_seen = None

            # change battery_charging to battery, and skup battery_discharging to battery
            fuel_tech = unit['fuel_tech']
            if fuel_tech == 'battery_charging':
                fuel_tech = 'battery'

            if fuel_tech != 'battery_discharging':
                rows.append({
                    'facility_code': facility['properties']['station_code'],
                    'duid': unit['duid'],
                    'network_region': unit['network_region'].lower(),
                    'capacity_registered': unit['capacity_registered'],
                    'fuel_tech': fuel_tech,
                    'status': unit['status'],
                    'data_first_seen': first_seen,
                    'data_last_seen': last_seen
                })
    df = pd.DataFrame(rows)

    # Ensure the 'data_first_seen' and 'data_last_seen' columns are treated as objects
    df['data_first_seen'] = df['data_first_seen'].astype('object')
    df['data_last_seen'] = df['data_last_seen'].astype('object')

    # clean up the data
    df['capacity_registered'] = df['capacity_registered'].astype(float)

    return df

all_units = build_unit_data_frame(raw_units_data)

all_units


,facility_code,duid,network_region,capacity_registered,fuel_tech,status,data_first_seen,data_last_seen
0,APPIN,APPIN,nsw1,55.0,gas_wcmg,operating,NaT,NaT
1,AVLSF,AVLSF1,nsw1,245.0,solar_utility,operating,NaT,2024-06-10 06:50:00
2,AWABAREF,AWABAREF,nsw1,1.0,bioenergy_biogas,retired,NaT,NaT
3,BANGOWF,BANGOWF2,nsw1,84.8,wind,operating,2021-06-04 14:25:00,2024-06-10 21:15:00
4,BANGOWF,BANGOWF1,nsw1,244.0,wind,operating,2020-11-30 14:25:00,2024-06-10 21:15:00
...,...,...,...,...,...,...,...,...
696,WARRADARGE,WARRADARGE_WF1,wem,180.0,wind,operating,2020-07-29 00:00:00,2024-01-13 23:55:00
697,BLAIRFOX_WESTHILLS,BLAIRFOX_WESTHILLS_WF3,wem,5.0,wind,operating,2013-10-10 00:00:00,2024-01-12 11:00:00
698,WEST_KALGOORLIE,WEST_KALGOORLIE_GT3,wem,23.3,distillate,operating,2006-09-19 16:00:00,2024-01-13 12:40:00
699,WEST_KALGOORLIE,WEST_KALGOORLIE_GT2,wem,41.2,distillate,operating,2006-09-19 16:00:00,2024-01-13 23:55:00


## check whether a unit was operating on a particular date

In [6]:
from datetime import date, timedelta
import unittest


def was_operating(status, first_seen, last_seen, given_date):
    month_start = date(given_date.year, given_date.month, 1)
    next_month = month_start + timedelta(days=31)  # Assuring to get to the next month
    month_end = date(next_month.year, next_month.month, 1) - timedelta(days=1)

    # print month_end formatted as YYYY-MM-DD HH:MM:SS
    # print("month_end:", month_end.strftime("%Y-%m-%d %H:%M:%S"))

    if status == "operating":
        return first_seen <= month_end
    elif status == "retired":
        return first_seen <= month_end and last_seen >= month_start
    elif status in {"commissioning", "committed"}:
        return False
    else:
        # throw an error
        raise ValueError(f"Unknown status: {status}")
    

def was_operating_unit(unit, given_date):
    return was_operating(unit['status'], unit['data_first_seen'], unit['data_last_seen'], given_date)


class TestWasOperating(unittest.TestCase):
    def setUp(self):
        self.unit = {
            "data_first_seen": date(1999, 2, 1),
            "data_last_seen": date(2002, 5, 1),
            "status": "retired",
            "fuel_tech": "solar_rooftop",
            "registered_capacity_MW": 50.2,
        }


    def test_was_operating_case1(self):
        self.assertTrue(
            was_operating_unit(self.unit, date(2000, 1, 1)), "Test Case 1 Failed"
        )


    def test_was_operating_case2(self):
        self.assertFalse(
            was_operating_unit(self.unit, date(2003, 6, 15)), "Test Case 2 Failed"
        )

## test cases for was_operating_unit

In [7]:
TEST_UNITS = [
    {
        "data_first_seen": date(1999, 2, 1),
        "data_last_seen": date(2002, 5, 31),
        "status": "retired",
        "fuel_tech": "solar_utility",
        "capacity_registered": 50,
        "network_region": "NSW1",
    },
    {
        "data_first_seen": date(2000, 6, 1),
        "data_last_seen": date(2005, 3, 1),
        "status": "retired",
        "fuel_tech": "solar_utility",
        "capacity_registered": 150,
        "network_region": "NSW1",
    },
    {
        "data_first_seen": date(2001, 4, 1),
        "data_last_seen": NaT,
        "status": "operating",
        "fuel_tech": "coal_black",
        "capacity_registered": 314.1,
        "network_region": "NSW1",
    },
    {
        "data_first_seen": date(1998, 12, 1),
        "data_last_seen": date(2020, 12, 1),
        "status": "retired",
        "fuel_tech": "coal_black",
        "capacity_registered": 350,
        "network_region": "NSW1",
    },
    {
        "data_first_seen": date(2002, 7, 1),
        "data_last_seen": NaT,
        "status": "operating",
        "fuel_tech": "hydro",
        "capacity_registered": 250.0,
        "network_region": "SA1",
    },
    {
        "data_first_seen": date(2003, 3, 1),
        "data_last_seen": NaT,
        "status": "committed",
        "fuel_tech": "hydro",
        "capacity_registered": 120,
        "network_region": "SA1",
    },
]


class TestWasOperating2(unittest.TestCase):
    def setUp(self):
        self.units = TEST_UNITS

    # Test cases for the first unit (retired)
    # unit[0] = {
    #     "data_first_seen": date(1999, 2, 1),
    #     "data_last_seen": date(2002, 5, 31),
    #     "status": "retired",
    #     "fuel_tech": "solar_rooftop",
    #     "capacity_registered": 50.2
    # }
    def test_unit1_before_first_seen(self):
        self.assertFalse(was_operating_unit(self.units[0], date(1998, 12, 31)))

    def test_unit1_during_operation(self):
        self.assertTrue(was_operating_unit(self.units[0], date(2000, 1, 1)))

    def test_unit1_after_last_seen(self):
        self.assertFalse(was_operating_unit(self.units[0], date(2003, 1, 1)))

    def test_unit1_after_last_seen_boundary_within(self):
        self.assertTrue(was_operating_unit(self.units[0], date(2002, 5, 31)))

    def test_unit1_after_last_seen_boundary_outside(self):
        self.assertFalse(was_operating_unit(self.units[0], date(2002, 6, 1)))

    def test_unit1_first_seen_boundary_within(self):
        self.assertTrue(was_operating_unit(self.units[0], date(1999, 2, 1)))

    def test_unit1_first_seen_boundary_outside(self):
        self.assertFalse(was_operating_unit(self.units[0], date(1999, 1, 31)))

    # Test cases for the second unit (retired)
    def test_unit2_before_first_seen(self):
        self.assertFalse(was_operating_unit(self.units[1], date(1999, 12, 31)))

    def test_unit2_during_operation(self):
        self.assertTrue(was_operating_unit(self.units[1], date(2002, 1, 1)))

    def test_unit2_after_last_seen(self):
        self.assertFalse(was_operating_unit(self.units[1], date(2006, 1, 1)))

    # Test cases for the third unit (operating, no end date)
    def test_unit3_before_first_seen(self):
        self.assertFalse(was_operating_unit(self.units[2], date(2000, 12, 31)))

    def test_unit3_after_first_seen(self):
        self.assertTrue(was_operating_unit(self.units[2], date(2002, 1, 1)))

    def test_unit3_future_date(self):
        self.assertTrue(was_operating_unit(self.units[2], date(2025, 1, 1)))

    # Test cases for the fourth unit (retired)
    def test_unit4_before_first_seen(self):
        self.assertFalse(was_operating_unit(self.units[3], date(1998, 11, 30)))

    def test_unit4_during_operation(self):
        self.assertTrue(was_operating_unit(self.units[3], date(2010, 1, 1)))

    def test_unit4_after_last_seen(self):
        self.assertFalse(was_operating_unit(self.units[3], date(2021, 1, 1)))

    # Test cases for the fifth unit (operating, no end date)
    def test_unit5_before_first_seen(self):
        self.assertFalse(was_operating_unit(self.units[4], date(2000, 6, 30)))

    def test_unit5_after_first_seen(self):
        self.assertTrue(was_operating_unit(self.units[4], date(2003, 1, 1)))

    def test_unit5_future_date(self):
        self.assertTrue(was_operating_unit(self.units[4], date(2025, 1, 1)))

    # Test cases for the sixth unit (committed/no end date)
    def test_unit6_before_first_seen(self):
        self.assertFalse(was_operating_unit(self.units[5], date(2002, 12, 31)))

    # unit6 = {
    #     "first_seen": date(2003, 3, 1),
    #     "status": "committed",
    #     "fuel_tech": "gas",
    #     "registered_capacity_MW": 120
    # }

    # failing
    def test_unit6_after_first_seen(self):
        self.assertFalse(was_operating_unit(self.units[5], date(2004, 1, 1)))

    # failing
    def test_unit6_future_date(self):
        self.assertFalse(was_operating_unit(self.units[5], date(2025, 1, 1)))


if __name__ == "__main__":
    unittest.main(argv=["first-arg-is-ignored"], verbosity=2, exit=False)

test_was_operating_case1 (__main__.TestWasOperating) ... ok
test_was_operating_case2 (__main__.TestWasOperating) ... ok
test_unit1_after_last_seen (__main__.TestWasOperating2) ... ok
test_unit1_after_last_seen_boundary_outside (__main__.TestWasOperating2) ... ok
test_unit1_after_last_seen_boundary_within (__main__.TestWasOperating2) ... ok
test_unit1_before_first_seen (__main__.TestWasOperating2) ... ok
test_unit1_during_operation (__main__.TestWasOperating2) ... ok
test_unit1_first_seen_boundary_outside (__main__.TestWasOperating2) ... ok
test_unit1_first_seen_boundary_within (__main__.TestWasOperating2) ... ok
test_unit2_after_last_seen (__main__.TestWasOperating2) ... ok
test_unit2_before_first_seen (__main__.TestWasOperating2) ... ok
test_unit2_during_operation (__main__.TestWasOperating2) ... ok
test_unit3_after_first_seen (__main__.TestWasOperating2) ... ok
test_unit3_before_first_seen (__main__.TestWasOperating2) ... ok
test_unit3_future_date (__main__.TestWasOperating2) ... ok


In [8]:
all_units

,facility_code,duid,network_region,capacity_registered,fuel_tech,status,data_first_seen,data_last_seen
0,APPIN,APPIN,nsw1,55.0,gas_wcmg,operating,NaT,NaT
1,AVLSF,AVLSF1,nsw1,245.0,solar_utility,operating,NaT,2024-06-10 06:50:00
2,AWABAREF,AWABAREF,nsw1,1.0,bioenergy_biogas,retired,NaT,NaT
3,BANGOWF,BANGOWF2,nsw1,84.8,wind,operating,2021-06-04 14:25:00,2024-06-10 21:15:00
4,BANGOWF,BANGOWF1,nsw1,244.0,wind,operating,2020-11-30 14:25:00,2024-06-10 21:15:00
...,...,...,...,...,...,...,...,...
696,WARRADARGE,WARRADARGE_WF1,wem,180.0,wind,operating,2020-07-29 00:00:00,2024-01-13 23:55:00
697,BLAIRFOX_WESTHILLS,BLAIRFOX_WESTHILLS_WF3,wem,5.0,wind,operating,2013-10-10 00:00:00,2024-01-12 11:00:00
698,WEST_KALGOORLIE,WEST_KALGOORLIE_GT3,wem,23.3,distillate,operating,2006-09-19 16:00:00,2024-01-13 12:40:00
699,WEST_KALGOORLIE,WEST_KALGOORLIE_GT2,wem,41.2,distillate,operating,2006-09-19 16:00:00,2024-01-13 23:55:00


## calculate the month ends

In [9]:
# given two dates, prepare an array with the dates of the last day of each month
# between the two dates (including the month of the first and last date)
def generate_month_ends(start_date, end_date):
    # Generate a complete range of month-end dates
    date_range = pd.date_range(start=start_date, end=end_date + pd.DateOffset(months=1), freq='ME')
    
    # Extract just the date part
    months = [date.date() for date in date_range]
    return months


st = find_first_seen(all_units)
en = date.today()
month_ends = generate_month_ends(st, en)

month_ends


[datetime.date(1998, 12, 31),
 datetime.date(1999, 1, 31),
 datetime.date(1999, 2, 28),
 datetime.date(1999, 3, 31),
 datetime.date(1999, 4, 30),
 datetime.date(1999, 5, 31),
 datetime.date(1999, 6, 30),
 datetime.date(1999, 7, 31),
 datetime.date(1999, 8, 31),
 datetime.date(1999, 9, 30),
 datetime.date(1999, 10, 31),
 datetime.date(1999, 11, 30),
 datetime.date(1999, 12, 31),
 datetime.date(2000, 1, 31),
 datetime.date(2000, 2, 29),
 datetime.date(2000, 3, 31),
 datetime.date(2000, 4, 30),
 datetime.date(2000, 5, 31),
 datetime.date(2000, 6, 30),
 datetime.date(2000, 7, 31),
 datetime.date(2000, 8, 31),
 datetime.date(2000, 9, 30),
 datetime.date(2000, 10, 31),
 datetime.date(2000, 11, 30),
 datetime.date(2000, 12, 31),
 datetime.date(2001, 1, 31),
 datetime.date(2001, 2, 28),
 datetime.date(2001, 3, 31),
 datetime.date(2001, 4, 30),
 datetime.date(2001, 5, 31),
 datetime.date(2001, 6, 30),
 datetime.date(2001, 7, 31),
 datetime.date(2001, 8, 31),
 datetime.date(2001, 9, 30),
 dateti

## build capacity table

In [10]:
def time_to_date(dt):
    if pd.isnull(dt):
        return None
    else:
        return dt.date()


def process_row(row, month_ends):
    capacity = row['capacity_registered']
    status = row['status']

    first_seen = time_to_date(row['data_first_seen'])
    last_seen = time_to_date(row['data_last_seen'])

    new_row = {
        'fuel_tech': row['fuel_tech'],
        'network_region': row['network_region']
    }

    if has_missing_dates(status, first_seen, last_seen):
        new_row['invalid_count'] = 1
        new_row['invalid_capacity'] = capacity
        for month_end in month_ends:
            new_row[month_end] = 0
    else:
        new_row['invalid_count'] = 0
        new_row['invalid_capacity'] = 0
        for month_end in month_ends:
            new_row[month_end] = capacity if was_operating(status, first_seen, last_seen, month_end) else 0

    return new_row


# construct the capacity table
def build_capacity_table(units, month_ends):
    processed_rows = [process_row(row, month_ends) for _, row in units.iterrows()]
    new_units = pd.DataFrame(processed_rows)

    new_units.info()

    # Make a summary table of units grouped by fuel_tech, network_region, summing up all the numeric columns
    summary = new_units.groupby(['fuel_tech', 'network_region']).sum()
    summary.reset_index(inplace=True)
    return summary

capacity_table = build_capacity_table(all_units, month_ends)

capacity_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Columns: 312 entries, fuel_tech to 2024-07-31
dtypes: float64(309), int64(1), object(2)
memory usage: 1.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Columns: 312 entries, fuel_tech to 2024-07-31
dtypes: float64(309), int64(1), object(2)
memory usage: 148.8+ KB


In [11]:
capacity_table

,fuel_tech,network_region,invalid_count,invalid_capacity,1998-12-31,1999-01-31,1999-02-28,1999-03-31,1999-04-30,1999-05-31,...,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,battery,nsw1,6,256.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,battery,qld1,3,186.0,0.0,0.0,0.0,0.0,0.0,0.0,...,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00
2,battery,sa1,2,300.0,0.0,0.0,0.0,0.0,0.0,0.0,...,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27
3,battery,tas1,1,28.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,battery,vic1,2,207.0,0.0,0.0,0.0,0.0,0.0,0.0,...,440.00,440.00,440.00,440.00,440.00,440.00,440.00,440.00,440.00,440.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,wind,qld1,3,348.0,0.0,0.0,0.0,0.0,0.0,0.0,...,675.20,675.20,675.20,675.20,675.20,675.20,675.20,675.20,675.20,675.20
57,wind,sa1,3,622.5,0.0,0.0,0.0,0.0,0.0,0.0,...,2140.90,2140.90,2140.90,2140.90,2140.90,2140.90,2140.90,2140.90,2140.90,2140.90
58,wind,tas1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,567.00,567.00,567.00,567.00,567.00,567.00,567.00,567.00,567.00,567.00
59,wind,vic1,12,371.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77


## calculate rooftop solar data

### load and cache rooftop solar data

In [12]:
APVI_BASE_URL = "https://pv-map.apvi.org.au/data/postcode/monthly/capacity/"
APVI_FILENAME = 'apvi_data.parquet'

def download_solar_PV_data():
    REGION_TO_POSTCODE = {
        "VIC1": "3XXX",
        "NSW1": "2XXX",
        "SA1": "5XXX",
        "WEM": "6XXX",
        "TAS1": "7XXX",
        "QLD1": "4XXX"
    }

    frames = []

    for region, postcode in REGION_TO_POSTCODE.items():
        url = APVI_BASE_URL + postcode
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data[postcode])
            df['network_region'] = region
            frames.append(df)
            print(f'INFO: region {region}: loaded {df.shape[0]} records from {df["month"].min()} to {df["month"].max()}')
        else:
             # die with an error
            raise Exception(f'ERROR: region {region} failed to load')

    df_all = pd.concat(frames)

    return df_all


def load_cached_solar_PV_data():
    INPUT_FILE = os.path.join(INPUT_FOLDER, APVI_FILENAME)

    # create if doesn't exist
    if not os.path.exists(INPUT_FILE):
        print(f'INFO: no cached data found, downloading from {APVI_BASE_URL}')
        downloaded_data = download_solar_PV_data()
        downloaded_data.to_parquet(INPUT_FILE)

    # load from cace
    df = pd.read_parquet(INPUT_FILE)
    print(f'INFO: loaded {df.shape[0]} records from {INPUT_FILE}')
    return df

solar = load_cached_solar_PV_data()
solar

INFO: no cached data found, downloading from https://pv-map.apvi.org.au/data/postcode/monthly/capacity/
INFO: region VIC1: loaded 207 records from 2007-01 to 2024-03
INFO: region NSW1: loaded 207 records from 2007-01 to 2024-03
INFO: region SA1: loaded 207 records from 2007-01 to 2024-03
INFO: region WEM: loaded 207 records from 2007-01 to 2024-03
INFO: region TAS1: loaded 203 records from 2007-04 to 2024-03
INFO: region QLD1: loaded 207 records from 2007-01 to 2024-03
INFO: loaded 1238 records from ./input/apvi_data.parquet


,month,2hf,2hf_4hf,4hf_6hf,6hf_9hf,9hf_14,14_25,25_50,50_100,100_5000,5000_30000,30000,network_region
0,2007-01,21.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VIC1
1,2007-02,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VIC1
2,2007-03,35.0,6.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VIC1
3,2007-04,43.0,12.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VIC1
4,2007-05,55.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VIC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2023-11,231.0,938.0,2567.0,19950.0,34574.0,6546.0,5270.0,4387.0,4463.0,0.0,0.0,QLD1
203,2023-12,148.0,784.0,2231.0,16713.0,30635.0,7935.0,8640.0,5099.0,1582.0,0.0,0.0,QLD1
204,2024-01,125.0,571.0,1725.0,12028.0,20174.0,2963.0,1649.0,858.0,1976.0,0.0,0.0,QLD1
205,2024-02,198.0,894.0,2307.0,17292.0,30253.0,4612.0,3352.0,299.0,0.0,0.0,0.0,QLD1


### process solar files

In [13]:
def get_last_day_of_month(date_str):
    # Convert the string to a Period with monthly frequency
    period = pd.Period(date_str, freq='M')
    
    # Convert the Period to a Timestamp representing the end of the month
    last_day_of_month = period.to_timestamp(how='end')
    
    # Extract and return the date part
    last_day_of_month = last_day_of_month.date()
    # print(f"changing {date_str} to {last_day_of_month}")
    return last_day_of_month


# sum up all capacity up to 5000 kW, convert months as text to end of month, and convert 'network_region' to lowercase
def process_data(df_all):
    # sum up all capacity up to 5000 kW
    df_all['capacity'] = df_all[['2hf', '2hf_4hf', '4hf_6hf', '6hf_9hf', '9hf_14', '14_25', '25_50', '50_100', '100_5000']].sum(axis=1)
    
    processed = df_all[['month', 'network_region', 'capacity']].copy()
    # df['month'] = pd.to_datetime(df['month'])

    # convert 'month' column to the end of the month
    processed['month'] = processed['month'].apply(get_last_day_of_month)

    # convert 'network_region' to lowercase
    processed['network_region'] = processed['network_region'].str.lower()

    return processed


small_scale_solar_build = process_data(solar)
small_scale_solar_build

,month,network_region,capacity
0,2007-01-31,vic1,24.0
1,2007-02-28,vic1,22.0
2,2007-03-31,vic1,46.0
3,2007-04-30,vic1,60.0
4,2007-05-31,vic1,60.0
...,...,...,...
202,2023-11-30,qld1,78926.0
203,2023-12-31,qld1,73767.0
204,2024-01-31,qld1,42069.0
205,2024-02-29,qld1,59207.0


### backfill missing months

In [14]:
# find any missing months and fill with zero new build capacity
def fix_missing_months(df, month_ends):
    print("INFO: ensuring that all {len(month_ends)} are filled for each region")

    # get all the unique network_regions in the data
    regions = df['network_region'].unique()

    new_rows = []

    for network_region in regions:
        region_df = small_scale_solar_build[small_scale_solar_build['network_region'] == network_region]

        # get just the months from tasmania
        region_months = set(region_df['month'].unique())

        # create a list of months in month_ends that are not in tasmania_months
        missing_months = list(set(month_ends) - region_months)
        missing_months.sort()

        print(f"INFO: {network_region} has data for {len(region_months)} months. filling in the missing {len(missing_months)} months")

        # add the missing months to the DataFrame
        for month in missing_months:
            new_rows.append(dict(network_region=network_region, month=month, capacity=0))
    
    # add the new rows to the DataFrame
    updated_df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    
    # reset the index
    updated_df = updated_df.reset_index(drop=True)
    
    return updated_df


filled = fix_missing_months(small_scale_solar_build, month_ends)

filled

# print("\nfixing missing months")
# data = fix_missing_months(data, month_ends)

# print("\ndata now complete")
# summarise(data)

# data


INFO: ensuring that all {len(month_ends)} are filled for each region
INFO: vic1 has data for 207 months. filling in the missing 101 months
INFO: nsw1 has data for 207 months. filling in the missing 101 months
INFO: sa1 has data for 207 months. filling in the missing 101 months
INFO: wem has data for 207 months. filling in the missing 101 months
INFO: tas1 has data for 203 months. filling in the missing 105 months
INFO: qld1 has data for 207 months. filling in the missing 101 months


,month,network_region,capacity
0,2007-01-31,vic1,24.0
1,2007-02-28,vic1,22.0
2,2007-03-31,vic1,46.0
3,2007-04-30,vic1,60.0
4,2007-05-31,vic1,60.0
...,...,...,...
1843,2006-12-31,qld1,0.0
1844,2024-04-30,qld1,0.0
1845,2024-05-31,qld1,0.0
1846,2024-06-30,qld1,0.0


### print a summary of the loaded PV data

In [15]:
from tabulate import tabulate
from IPython.display import Markdown, display

def print_markdown_table(df):
    # Create a pivot table
    pivot_table = df.pivot_table(index='network_region', 
                                 values='capacity', 
                                 aggfunc=['sum', 'count']).reset_index()
    
    # Flatten the column MultiIndex created by pivot_table
    pivot_table.columns = ['Region', 'Row Count', 'Capacity (kW)']

    # Calculate the total capacity and total count of rows
    total_capacity = pivot_table['Capacity (kW)'].sum()
    total_count = pivot_table['Row Count'].sum()

    # Append the total row to the DataFrame
    total_row = pd.DataFrame([['Total', total_count, total_capacity]], 
                             columns=['Region', 'Row Count', 'Capacity (kW)'])
    pivot_table = pd.concat([pivot_table, total_row], ignore_index=True)

    # Format the capacity values to 0,000 format
    pivot_table['Capacity (kW)'] = pivot_table['Capacity (kW)'].apply(lambda x: f"{x:,.0f}")
    pivot_table['Row Count'] = pivot_table['Row Count'].apply(lambda x: f"{x:,.0f}")

    # Use tabulate to create a markdown table
    markdown_table = tabulate(pivot_table, headers='keys', tablefmt='github', showindex=False)
    
    # Display the markdown table
    display(Markdown(markdown_table))
    

# Print the markdown table
print_markdown_table(filled)

| Region   | Row Count   | Capacity (kW)   |
|----------|-------------|-----------------|
| nsw1     | 6,998,011   | 308             |
| qld1     | 6,333,083   | 308             |
| sa1      | 2,545,664   | 308             |
| tas1     | 315,705     | 308             |
| vic1     | 4,701,939   | 308             |
| wem      | 2,694,285   | 308             |
| Total    | 23,588,687  | 1,848           |

### compute cumulative capacity

In [16]:
def cumulative_capacity(df):
    cumulative_capacity = df.copy()

    # first sort by month then region
    cumulative_capacity.sort_values(by=['month', 'network_region'], inplace=True)

    # then groupby month and region and cumsum the capacity
    cumulative_capacity['cumulative_capacity'] = cumulative_capacity.groupby('network_region')['capacity'].transform(pd.Series.cumsum)
    return cumulative_capacity


# all_regions = add_aggregated_region(filled)

cumulative = cumulative_capacity(filled)


cumulative.info()
cumulative

<class 'pandas.core.frame.DataFrame'>
Index: 1848 entries, 1339 to 1641
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   month                1848 non-null   object 
 1   network_region       1848 non-null   object 
 2   capacity             1848 non-null   float64
 3   cumulative_capacity  1848 non-null   float64
dtypes: float64(2), object(2)
memory usage: 72.2+ KB


,month,network_region,capacity,cumulative_capacity
1339,1998-12-31,nsw1,0.0,0.0
1747,1998-12-31,qld1,0.0,0.0
1440,1998-12-31,sa1,0.0,0.0
1642,1998-12-31,tas1,0.0,0.0
1238,1998-12-31,vic1,0.0,0.0
...,...,...,...,...
1847,2024-07-31,qld1,0.0,6333083.0
1540,2024-07-31,sa1,0.0,2545664.0
1746,2024-07-31,tas1,0.0,315705.0
1338,2024-07-31,vic1,0.0,4701939.0


### create a pivot table of solar_rooftop in MW

In [17]:
def create_solar_capacity_table(combined_table):
    regions = {}

    # prime the output table
    for region in {'qld1', 'nsw1', 'vic1', 'sa1', 'tas1', 'wem'}:
        regions[region] = {
        'fuel_tech': 'solar_rooftop',
        'network_region': region,
        'invalid_count': 0,
        'invalid_capacity': 0
    }
    
    # iterate through the combined_table, and for each row, add the capacity to the correct region
    for _, row in combined_table.iterrows():
        region = regions[row['network_region']]
        region[row['month']] = row['cumulative_capacity'] / 1000

    # convert the region_data to a dataframe
    df = pd.DataFrame(regions).transpose()

    # sort by fuel_tech then network_region
    df = df.sort_values(by=['fuel_tech', 'network_region'])

    # reindex without the index column
    df = df.reset_index()
    df = df.drop(columns=['index'])
    
    return df

    
rooftop_capacity_table = create_solar_capacity_table(cumulative)
rooftop_capacity_table


,fuel_tech,network_region,invalid_count,invalid_capacity,1998-12-31,1999-01-31,1999-02-28,1999-03-31,1999-04-30,1999-05-31,...,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,solar_rooftop,nsw1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,6578.324,6687.923,6799.86,6864.016,6942.435,6998.011,6998.011,6998.011,6998.011,6998.011
1,solar_rooftop,qld1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,6038.546,6117.472,6191.239,6233.308,6292.515,6333.083,6333.083,6333.083,6333.083,6333.083
2,solar_rooftop,sa1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,2456.565,2483.698,2508.045,2519.888,2536.352,2545.664,2545.664,2545.664,2545.664,2545.664
3,solar_rooftop,tas1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,297.704,303.306,307.848,310.092,313.568,315.705,315.705,315.705,315.705,315.705
4,solar_rooftop,vic1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,4502.021,4563.727,4621.912,4652.376,4687.728,4701.939,4701.939,4701.939,4701.939,4701.939
5,solar_rooftop,wem,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,2585.92,2612.518,2638.291,2657.044,2680.393,2694.285,2694.285,2694.285,2694.285,2694.285


## merge rooftop_solar into with all facilities

In [18]:
# merge rooftop solar with the master capacity table
def merge_tables(facilities_table, rooftop_table):
    # first check that the columns are the same
    if not facilities_table.columns.equals(rooftop_table.columns):
        print("ERROR: columns are not the same")
        print(f"facilities columns: {facilities_table.columns}")
        print(f"rooftop columns: {rooftop_table.columns}")
        raise Exception("Columns are not the same")

    # concatenate the two tables
    unified_table = pd.concat([facilities_table, rooftop_table], axis=0, ignore_index=True)

    # sort by fuel_tech then network_region
    unified_table = unified_table.sort_values(by=['fuel_tech', 'network_region'])

    # reindex the table
    unified_table = unified_table.reset_index(drop=True)
    return unified_table


facilities_with_rooftop_table = merge_tables(capacity_table, rooftop_capacity_table)
facilities_with_rooftop_table

,fuel_tech,network_region,invalid_count,invalid_capacity,1998-12-31,1999-01-31,1999-02-28,1999-03-31,1999-04-30,1999-05-31,...,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,battery,nsw1,6,256.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,battery,qld1,3,186.0,0.0,0.0,0.0,0.0,0.0,0.0,...,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0
2,battery,sa1,2,300.0,0.0,0.0,0.0,0.0,0.0,0.0,...,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27
3,battery,tas1,1,28.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,battery,vic1,2,207.0,0.0,0.0,0.0,0.0,0.0,0.0,...,440.0,440.0,440.0,440.0,440.0,440.0,440.0,440.0,440.0,440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,wind,qld1,3,348.0,0.0,0.0,0.0,0.0,0.0,0.0,...,675.2,675.2,675.2,675.2,675.2,675.2,675.2,675.2,675.2,675.2
63,wind,sa1,3,622.5,0.0,0.0,0.0,0.0,0.0,0.0,...,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9
64,wind,tas1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,567.0,567.0,567.0,567.0,567.0,567.0,567.0,567.0,567.0,567.0
65,wind,vic1,12,371.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77


In [19]:
facilities_with_rooftop_table

,fuel_tech,network_region,invalid_count,invalid_capacity,1998-12-31,1999-01-31,1999-02-28,1999-03-31,1999-04-30,1999-05-31,...,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,battery,nsw1,6,256.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,battery,qld1,3,186.0,0.0,0.0,0.0,0.0,0.0,0.0,...,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0
2,battery,sa1,2,300.0,0.0,0.0,0.0,0.0,0.0,0.0,...,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27
3,battery,tas1,1,28.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,battery,vic1,2,207.0,0.0,0.0,0.0,0.0,0.0,0.0,...,440.0,440.0,440.0,440.0,440.0,440.0,440.0,440.0,440.0,440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,wind,qld1,3,348.0,0.0,0.0,0.0,0.0,0.0,0.0,...,675.2,675.2,675.2,675.2,675.2,675.2,675.2,675.2,675.2,675.2
63,wind,sa1,3,622.5,0.0,0.0,0.0,0.0,0.0,0.0,...,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9
64,wind,tas1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,567.0,567.0,567.0,567.0,567.0,567.0,567.0,567.0,567.0,567.0
65,wind,vic1,12,371.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77


## add in NEM summary

In [20]:
# function that sums the data, grouping by fuel_tech, only where network_region is qld1, nsw1, vic1, tas1 and sa1
# then, with the result, adds a network_region column = "_all"
def create_summary(df, regions, summary_name):
    # filter the DataFrame using the isin method
    df_NEM = df[df['network_region'].isin(regions)]

    summed = df_NEM.groupby(['fuel_tech']).sum().reset_index()
    summed['network_region'] = summary_name
    return summed


# create a new table that concatenates nem_table and capacity_table
def create_and_merge_summaries(capacity_table):
    nem_table = create_summary(capacity_table, ['qld1', 'nsw1', 'vic1', 'tas1', 'sa1'], '_nem')
    au_table = create_summary(capacity_table, ['qld1', 'nsw1', 'vic1', 'tas1', 'sa1', 'wem'], '_au')

    # Ensure indexes are reset
    nem_table.reset_index(drop=True, inplace=True)
    capacity_table.reset_index(drop=True, inplace=True)
    au_table.reset_index(drop=True, inplace=True)

    # concatenate if columns are identical
    if set(nem_table.columns) != set(capacity_table.columns) and set(nem_table.columns) != set(au_table.columns):
        raise ValueError("DataFrames do not have identical columns and cannot be concatenated.")

    combined_table = pd.concat([nem_table, au_table, capacity_table], ignore_index=True)

    # sort by fuel_tech then network_region
    combined_table.sort_values(by=['fuel_tech', 'network_region'], inplace=True)
    combined_table.reset_index(drop=True, inplace=True)
    
    return combined_table


unified_table = create_and_merge_summaries(facilities_with_rooftop_table)
unified_table


,fuel_tech,network_region,invalid_count,invalid_capacity,1998-12-31,1999-01-31,1999-02-28,1999-03-31,1999-04-30,1999-05-31,...,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,battery,_au,14,977.0,0.0,0.0,0.0,0.0,0.0,0.0,...,801.27,801.27,801.27,801.27,801.27,801.27,801.27,801.27,801.27,801.27
1,battery,_nem,14,977.0,0.0,0.0,0.0,0.0,0.0,0.0,...,801.27,801.27,801.27,801.27,801.27,801.27,801.27,801.27,801.27,801.27
2,battery,nsw1,6,256.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,battery,qld1,3,186.0,0.0,0.0,0.0,0.0,0.0,0.0,...,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0
4,battery,sa1,2,300.0,0.0,0.0,0.0,0.0,0.0,0.0,...,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27,211.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,wind,qld1,3,348.0,0.0,0.0,0.0,0.0,0.0,0.0,...,675.2,675.2,675.2,675.2,675.2,675.2,675.2,675.2,675.2,675.2
95,wind,sa1,3,622.5,0.0,0.0,0.0,0.0,0.0,0.0,...,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9,2140.9
96,wind,tas1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,567.0,567.0,567.0,567.0,567.0,567.0,567.0,567.0,567.0,567.0
97,wind,vic1,12,371.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77,2643.77


In [21]:
# find any missing months and fill with zero new build capacity
def fix_missing_months(df, month_ends):
    print("INFO: ensuring that all {len(month_ends)} are filled for each region")

    # get all the unique network_regions in the data
    regions = df['network_region'].unique()

    new_rows = []

    for network_region in regions:
        region_df = small_scale_solar_build[small_scale_solar_build['network_region'] == network_region]

        # get just the months from tasmania
        region_months = set(region_df['month'].unique())

        # create a list of months in month_ends that are not in tasmania_months
        missing_months = list(set(month_ends) - region_months)
        missing_months.sort()

        print(f"INFO: {network_region} has data for {len(region_months)} months. filling in the missing {len(missing_months)} months")

        # add the missing months to the DataFrame
        for month in missing_months:
            new_rows.append(dict(network_region=network_region, month=month, capacity=0))
    
    # add the new rows to the DataFrame
    updated_df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    
    # reset the index
    updated_df = updated_df.reset_index(drop=True)
    
    return updated_df


filled = fix_missing_months(small_scale_solar_build, month_ends)

filled

# print("\nfixing missing months")
# data = fix_missing_months(data, month_ends)

# print("\ndata now complete")
# summarise(data)

# data


INFO: ensuring that all {len(month_ends)} are filled for each region
INFO: vic1 has data for 207 months. filling in the missing 101 months
INFO: nsw1 has data for 207 months. filling in the missing 101 months
INFO: sa1 has data for 207 months. filling in the missing 101 months
INFO: wem has data for 207 months. filling in the missing 101 months
INFO: tas1 has data for 203 months. filling in the missing 105 months
INFO: qld1 has data for 207 months. filling in the missing 101 months


,month,network_region,capacity
0,2007-01-31,vic1,24.0
1,2007-02-28,vic1,22.0
2,2007-03-31,vic1,46.0
3,2007-04-30,vic1,60.0
4,2007-05-31,vic1,60.0
...,...,...,...
1843,2006-12-31,qld1,0.0
1844,2024-04-30,qld1,0.0
1845,2024-05-31,qld1,0.0
1846,2024-06-30,qld1,0.0


## create region files

In [22]:
import re

# put all the elements of the numerical array on the same line
def compact_json(json_str):
    # regex looks for a _numerical_ data array and captures everything in between [ and ]
    pattern = r'("data": \[)([\d\., \n]*)(\])'
    
    # inner function is used to replace newlines and spaces between array items
    def replacer(match):
        start, middle, end = match.groups()
        # Replace newline and possible spaces
        compact_middle = middle.replace('\n', '').replace(' ', '').replace(',', ', ')
        return start + compact_middle + end
    
    return re.sub(pattern, replacer, json_str, flags=re.DOTALL)


def generate_id(region, fuel_tech):
    if region == '_au':
        id = "au._au."
    elif region == '_nem':
        id = "au.nem._nem."
    elif region == 'wem':
        id = "au.wem."
    else:
        id = f"au.nem.{region}."

    return id + f"fuel_tech.{fuel_tech}.capacity"


# pull out all the column names that are date objects
def get_date_columns(df):
    # cols = [col for col in df.columns if isinstance(df[col][0], datetime)]
    cols = [col for col in df.columns if pd.to_datetime(col, errors='coerce', format='%Y-%m-%d') is not pd.NaT]

    # check that there's at least one date column
    if len(cols) == 0:
        df.info()
        raise ValueError("ERROR: can't have no date columns!")
    
    # check that all are in date order
    if not all(earlier <= later for earlier, later in zip(cols, cols[1:])):
        raise ValueError("ERROR: date columns are not in date order")
    
    # we're either going to be looking at months or years
    # if months, dates could be 28-31 days apart
    # if years, dates could be 365-366 days apart
    # so we'll look at the difference between max and min intervals, and it should be <= 3
    deltas = [(date2 - date1).days for date1, date2 in zip(cols[:-1], cols[1:])]
    range = max(deltas) - min(deltas)
    if range > 3:
        raise ValueError("ERROR: date columns appear to have a gap")

    return cols


def buildJSON(capacity_table, region):
    data = []

    # extract start and end month
    date_cols = get_date_columns(capacity_table)

    start = f"{date_cols[0]}"
    last = f"{date_cols[-1]}"
   
    for i, row in capacity_table.iterrows():
        # change numbers to just 1 decimal place
        capacities = [round(x, 1) for x in row[date_cols]]

        if region == '_au':
            net_name = None
            reg_name = None
        elif region == 'wem':
            net_name = 'wem'
            reg_name = None
        elif region == '_nem':
            net_name = 'nem'
            reg_name = '_nem'
        else: # every NEM region
            net_name = 'nem'
            reg_name = region

        element = {
            "id": generate_id(row['network_region'], row['fuel_tech']),
            "type": 'capacity',
            "network": net_name,
            "region": reg_name,
            "fuel_tech": row['fuel_tech'],
            "units": "MW",
            "invalid_count": row['invalid_count'],
            "invalid_capacity": round(row['invalid_capacity'], 1),
            "history": {
                "start": start,
                "last": last,
                "interval": "1M",
                "data": capacities
            }
        }

        # if network or region is none, delete it
        if net_name is None:
            del element['network']

        if reg_name is None:
            del element['region'] 
            
        data.append(element)

    output_obj = {
        "version": "4.1",
        "network": net_name,
        "region": reg_name,
        "created_at": datetime.now(TIME_ZONE).strftime("%Y-%m-%dT%H:%M:%S%z"),
        "messages": [ "historical capacity derived from facilities data" ],
        "data": data,
    }

    # if network or region is none, delete it
    if net_name is None:
        del output_obj['network']

    if reg_name is None:
        del output_obj['region'] 

    json_output = json.dumps(output_obj, indent=2)
    return compact_json(json_output)


def get_folder(base_folder,region):
    if region == '_au':
        net_name = None
        reg_name = None
    elif region == 'wem':
        net_name = 'wem'
        reg_name = None
    elif region == '_nem':
        net_name = 'nem'
        reg_name = '_nem'
    else: # every NEM region
        net_name = 'nem'
        reg_name = region

    path = os.path.join(base_folder, "historical_capacity", "au")

    if net_name:
        path = os.path.join(path, net_name)

    if reg_name and reg_name != '_nem':
        path = os.path.join(path, reg_name)

    return path
    

# create region file
def create_region_file(capacity_table, region, output_dir, file_name):
    folder = get_folder(output_dir, region)
    os.makedirs(folder, exist_ok=True)

    output_file = os.path.join(folder, file_name)
    print(f"INFO: writing capacity file for {region} to {output_file}")

    json_output = buildJSON(capacity_table, region)

    with open(output_file, "w") as f:
        f.write(json_output)


# for each region in combined_table, call a function with all rows in that region
def create_region_files(frame, output_dir, file_name):
    output_dir = os.path.join(output_dir)

    print(f"INFO: creating folder {output_dir}")
    os.makedirs(output_dir, exist_ok=True)

    # get the list of regions from combined_table
    regions = unified_table['network_region'].unique()

    # iterate through each region
    for region in regions:
        # filter the combined_table to only include rows with the current region
        region_table = frame[frame['network_region'] == region]
        create_region_file(region_table, region, output_dir, file_name)


create_region_files(unified_table, OUTPUT_FOLDER, 'monthly.json')


INFO: creating folder ./output
INFO: writing capacity file for _au to ./output/historical_capacity/au/monthly.json
INFO: writing capacity file for _nem to ./output/historical_capacity/au/nem/monthly.json
INFO: writing capacity file for nsw1 to ./output/historical_capacity/au/nem/nsw1/monthly.json
INFO: writing capacity file for qld1 to ./output/historical_capacity/au/nem/qld1/monthly.json
INFO: writing capacity file for sa1 to ./output/historical_capacity/au/nem/sa1/monthly.json
INFO: writing capacity file for tas1 to ./output/historical_capacity/au/nem/tas1/monthly.json
INFO: writing capacity file for vic1 to ./output/historical_capacity/au/nem/vic1/monthly.json
INFO: writing capacity file for wem to ./output/historical_capacity/au/wem/monthly.json


In [23]:
def get_annual_capacity(df):
    my_dates = get_date_columns(unified_table)

    # filter to just 30 june
    june_dates = [date for date in my_dates if date.month == 6 and date.day == 30]

    # copy unified_table, just the columns fuel_tech, network_region, invalid_count, invalid_capacity and june_dates
    annual_table = unified_table[['fuel_tech', 'network_region', 'invalid_count', 'invalid_capacity', *june_dates]].copy()

    new_dates = get_date_columns(annual_table)

    # the capacity at the end of june is the capacity at the beginning of july
    new_column_names = {date: date + timedelta(days=1) for date in new_dates}
    annual_table.rename(columns=new_column_names, inplace=True)

    return annual_table


annual_table = get_annual_capacity(unified_table)
create_region_files(annual_table, OUTPUT_FOLDER, 'annual.json')


INFO: creating folder ./output
INFO: writing capacity file for _au to ./output/historical_capacity/au/annual.json
INFO: writing capacity file for _nem to ./output/historical_capacity/au/nem/annual.json
INFO: writing capacity file for nsw1 to ./output/historical_capacity/au/nem/nsw1/annual.json
INFO: writing capacity file for qld1 to ./output/historical_capacity/au/nem/qld1/annual.json
INFO: writing capacity file for sa1 to ./output/historical_capacity/au/nem/sa1/annual.json
INFO: writing capacity file for tas1 to ./output/historical_capacity/au/nem/tas1/annual.json
INFO: writing capacity file for vic1 to ./output/historical_capacity/au/nem/vic1/annual.json
INFO: writing capacity file for wem to ./output/historical_capacity/au/wem/annual.json
